In [ ]:
"""
Wallet Pipeline – Fetch + Metrics (Jupyter Edition)
===================================================

End‑to‑end notebook‑style script that:

1. **Downloads** raw transaction data for Elliptic wallets (class 2) from
   *Blockchain.info* using a rotating proxy and resilient retries.
2. **Validates** all JSON files on disk, discarding corrupt/empty ones.
3. **Computes per‑wallet graph & temporal metrics** and writes the final CSV
   `elliptic_licit.csv` ready for downstream modelling.

> Each `# %%` delimiter denotes a cell when opened in Jupyter Lab/Notebook or
> VS Code.  Run cells top‑to‑bottom.  Adapt constants in *Configuration* as
> needed.
"""

In [ ]:
from __future__ import annotations

import csv
import json
import logging
import os
import re
import sys
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from pathlib import Path
from typing import List, Sequence

import pandas as pd
import requests
from tqdm.auto import tqdm

# -----------------------------------------------------------------------------
# Configuration – tweak as needed
# -----------------------------------------------------------------------------
DATA_FILE = "wallets_classes.csv"      # CSV with `address,class` columns
CLASS_FILTER = 2                       # Elliptic: 2 = licit

OUTPUT_DIR = Path("wallets/elliptic_licit")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PROXY_URL = os.getenv("ROTATING_PROXY_URL", "http://YOUR_ROTATING_PROXY")

MAX_WORKERS = 5            # parallel threads for download
BATCH_SIZE = 50            # wallets per batch
DELAY_BETWEEN_BATCHES = 2  # s between batches
REQUEST_TIMEOUT = 30       # HTTP timeout
MAX_RETRIES = 3            # per‑request retries
BACKOFF_FACTOR = 1.5       # exponential back‑off multiplier

# Logging ---------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
)
logger = logging.getLogger("wallet-pipeline")

In [ ]:
# -----------------------------------------------------------------------------
# Fetch helpers – resilient HTTP, skip already‑downloaded wallets
# -----------------------------------------------------------------------------

def load_wallets(csv_path: str, wallet_class: int) -> List[str]:
    """Return wallet addresses whose *class* equals *wallet_class*."""
    with open(csv_path, newline="") as fh:
        reader = csv.DictReader(fh)
        return [row["address"] for row in reader if int(row["class"]) == wallet_class]


def already_downloaded(out_dir: Path) -> set[str]:
    pattern = re.compile(r"(.+)_transactions\.json$")
    return {
        m.group(1)
        for p in out_dir.glob("*_transactions.json")
        if (m := pattern.match(p.name))
    }


def _request_with_retry(session: requests.Session, url: str, retries: int = MAX_RETRIES):
    attempt = 0
    while attempt <= retries:
        try:
            resp = session.get(url, timeout=REQUEST_TIMEOUT)
            if resp.status_code == 200:
                return resp
            if resp.status_code in (429, 500, 502, 503, 504):
                delay = BACKOFF_FACTOR ** attempt
                logger.warning("HTTP %s – retrying in %.1fs", resp.status_code, delay)
                time.sleep(delay)
            else:
                logger.error("Non‑recoverable status %s", resp.status_code)
                return None
        except requests.RequestException as exc:
            delay = BACKOFF_FACTOR ** attempt
            logger.warning("%s – retrying in %.1fs", type(exc).__name__, delay)
            time.sleep(delay)
        attempt += 1
    return None


def fetch_and_save(wallet: str, out_dir: Path, session: requests.Session) -> bool:
    url = f"https://blockchain.info/rawaddr/{wallet}"
    resp = _request_with_retry(session, url)
    if resp and resp.status_code == 200:
        out_path = out_dir / f"{wallet}_transactions.json"
        out_path.write_text(json.dumps(resp.json(), indent=2))
        logger.info("✅ Saved %s", out_path.name)
        return True
    logger.error("❌ Failed %s", wallet)
    return False


def download_wallets(wallets: Sequence[str], out_dir: Path) -> list[str]:
    """Download *wallets* and return the list of failed addresses."""
    failed: list[str] = []
    proxy_dict = {"http": PROXY_URL, "https": PROXY_URL} if PROXY_URL else None
    with requests.Session() as session:
        if proxy_dict:
            session.proxies.update(proxy_dict)

        for batch_no, start in enumerate(range(0, len(wallets), BATCH_SIZE), 1):
            batch = wallets[start : start + BATCH_SIZE]
            logger.info("Batch %s/%s – %s wallets", batch_no, (len(wallets) - 1)//BATCH_SIZE + 1, len(batch))
            with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
                futures = {pool.submit(fetch_and_save, w, out_dir, session): w for w in batch}
                for fut in as_completed(futures):
                    if not fut.result():
                        failed.append(futures[fut])
            if start + BATCH_SIZE < len(wallets):
                logger.info("Sleeping %ss…", DELAY_BETWEEN_BATCHES)
                time.sleep(DELAY_BETWEEN_BATCHES)
    return failed

In [ ]:
# -----------------------------------------------------------------------------
# Execute download phase
# -----------------------------------------------------------------------------
wallet_all = load_wallets(DATA_FILE, CLASS_FILTER)
wallet_done = already_downloaded(OUTPUT_DIR)
wallet_queue = [w for w in wallet_all if w not in wallet_done]
logger.info("%s / %s wallets already on disk", len(wallet_done), len(wallet_all))
logger.info("Queueing %s wallets for download", len(wallet_queue))

failed_wallets = download_wallets(wallet_queue, OUTPUT_DIR)
if failed_wallets:
    (OUTPUT_DIR / "failed_wallets.json").write_text(json.dumps(sorted(failed_wallets), indent=2))
    logger.warning("Wrote failed_wallets.json containing %s addresses", len(failed_wallets))
logger.info("Download finished – success: %s, failed: %s", len(wallet_queue) - len(failed_wallets), len(failed_wallets))

In [ ]:
# -----------------------------------------------------------------------------
# Validation – build list of JSON files with a non‑empty 'txs' field
# -----------------------------------------------------------------------------

def get_wallet_transactions(file_path: str):
    try:
        with open(file_path, "r") as fh:
            data = json.load(fh)
        return data.get("txs", [])
    except Exception as exc:
        logger.error("Invalid JSON %s – %s", file_path, exc)
        return []


def list_valid_files(base_dir: Path) -> list[str]:
    files = [p for p in base_dir.glob("*.json") if p.is_file()]
    valid: list[str] = []
    with ThreadPoolExecutor() as pool:
        futures = {pool.submit(get_wallet_transactions, str(p)): p for p in files}
        for fut in tqdm(as_completed(futures), total=len(files), desc="Validating JSON"):
            if fut.result():
                valid.append(str(futures[fut]))
    logger.info("%s / %s JSON files contain transactions", len(valid), len(files))
    return valid

valid_files = list_valid_files(OUTPUT_DIR)

In [ ]:
# -----------------------------------------------------------------------------
# Metrics helpers
# -----------------------------------------------------------------------------

def calculate_all_metrics(transactions: list[dict]) -> pd.DataFrame:
    """Return a DataFrame with one row per (output_address, input_address) pair."""
    records = []
    for tx in transactions:
        tx_time = datetime.fromtimestamp(tx["time"])
        inputs = [inp["prev_out"].get("addr") for inp in tx["inputs"] if inp["prev_out"].get("addr")]
        for out in tx["out"]:
            out_addr = out.get("addr")
            if out_addr:
                for in_addr in inputs:
                    records.append({
                        "input_address": in_addr,
                        "output_address": out_addr,
                        "amount": out["value"] / 1e8,
                        "time": tx_time,
                    })
    df = pd.DataFrame(records)
    if df.empty:
        return df

    df.sort_values("time", inplace=True)
    df["time_diff"] = df.groupby("output_address")["time"].diff().dt.total_seconds() / 86400

    # Basic aggregates ---------------------------------------------------------
    grp_out = df.groupby("output_address")
    grp_in  = df.groupby("input_address")

    out_deg      = grp_out.size()
    in_deg       = grp_in.size()
    uniq_in_deg  = grp_out["input_address"].nunique()
    uniq_out_deg = grp_in["output_address"].nunique()
    avg_in_amt   = grp_out["amount"].mean()
    avg_out_amt  = grp_in["amount"].mean()
    avg_in_time  = grp_out["time_diff"].mean()
    avg_out_time = grp_in["time_diff"].mean()
    total_in_amt = grp_out["amount"].sum()
    total_out_amt= grp_in["amount"].sum()

    # Balance & clustering -----------------------------------------------------
    balance = (total_in_amt - total_out_amt.reindex(total_in_amt.index, fill_value=0))
    clust_coeff = out_deg / (in_deg.reindex(out_deg.index, fill_value=0) + 1)

    # Active duration ----------------------------------------------------------
    active_dur = (grp_out["time"].max() - grp_out["time"].min()).dt.total_seconds() / 86400

    metrics = pd.DataFrame({
        "in_degree": out_deg,
        "out_degree": in_deg.reindex(out_deg.index, fill_value=0),
        "unique_in_degree": uniq_in_deg,
        "unique_out_degree": uniq_out_deg.reindex(out_deg.index, fill_value=0),
        "avg_in_transaction": avg_in_amt,
        "avg_out_transaction": avg_out_amt.reindex(out_deg.index, fill_value=0),
        "avg_in_time_interval": avg_in_time,
        "avg_out_time_interval": avg_out_time.reindex(out_deg.index, fill_value=0),
        "balance": balance,
        "clustering_coefficient": clust_coeff,
        "active_duration": active_dur,
    })
    return metrics


def aggregate_wallet_metrics(m: pd.DataFrame) -> pd.DataFrame:
    """Aggregate per‑address metrics to a single‑row wallet‑level DataFrame."""
    s = {
        "total_in_degree": m["in_degree"].sum(),
        "total_out_degree": m["out_degree"].sum(),
        "total_unique_in": m["unique_in_degree"].sum(),
        "total_unique_out": m["unique_out_degree"].sum(),
        "avg_in_transaction": m["avg_in_transaction"].mean(),
        "avg_out_transaction": m["avg_out_transaction"].mean(),
        "total_received": m["balance"].clip(lower=0).sum(),
        "total_sent": (-m["balance"].clip(upper=0)).sum(),
        "net_balance": m["balance"].sum(),
        "avg_in_time_interval": m["avg_in_time_interval"].mean(),
        "avg_out_time_interval": m["avg_out_time_interval"].mean(),
        "avg_active_duration": m["active_duration"].mean(),
    }
    return pd.DataFrame([s])


def compute_combined_metrics(w: pd.DataFrame) -> pd.DataFrame:
    ε = 1e-8
    r = w.iloc[0]
    combined = {
        "in_out_ratio": r["total_in_degree"] / (r["total_out_degree"] + ε),
        "unique_in_ratio": r["total_unique_in"] / (r["total_in_degree"] + ε),
        "unique_out_ratio": r["total_unique_out"] / (r["total_out_degree"] + ε),
        "volume_ratio": r["total_received"] / (r["total_sent"] + ε),
        "net_balance_ratio": r["net_balance"] / (r["total_received"] + ε),
        "activity_index": (r["total_in_degree"] + r["total_out_degree"]) / (r["avg_active_duration"] + ε),
        "time_interval_ratio": r["avg_out_time_interval"] / (r["avg_in_time_interval"] + ε),
        "weighted_avg_tx": (
            (r["avg_in_transaction"] * r["total_in_degree"]) +
            (r["avg_out_transaction"] * r["total_out_degree"])
        ) / (r["total_in_degree"] + r["total_out_degree"] + ε),
    }
    return pd.DataFrame([combined])


def process_wallet(file_path: str):
    txs = get_wallet_transactions(file_path)
    if not txs:
        return None
    m = calculate_all_metrics(txs)
    if m.empty:
        return None
    base = aggregate_wallet_metrics(m)
    comb = compute_combined_metrics(base)

    times = [datetime.fromtimestamp(tx["time"]) for tx in txs]
    created, last = min(times), max(times)

    df_wallet = pd.concat([base, comb], axis=1)
    df_wallet["wallet_file"] = file_path
    df_wallet["wallet_creation_ts"] = created.timestamp()
    df_wallet["wallet_last_tx_ts"] = last.timestamp()
    df_wallet["wallet_lifetime_sec"] = (last - created).total_seconds()
    return df_wallet

In [ ]:
# -----------------------------------------------------------------------------
# Run metric computation
# -----------------------------------------------------------------------------
results: list[pd.DataFrame] = []
for fp in tqdm(valid_files, desc="Computing metrics"):
    row = process_wallet(fp)
    if row is not None:
        results.append(row)

if not results:
    logger.error("No metrics could be computed – aborting")
    raise SystemExit

metrics_df = pd.concat(results, ignore_index=True)
logger.info("Final metrics DataFrame shape: %s rows x %s cols", *metrics_df.shape)

# Clean up columns -------------------------------------------------------------
metrics_df["wallet"] = metrics_df["wallet_file"].apply(lambda p: Path(p).stem.split("_")[0])
metrics_df.drop(columns=["wallet_file"], inplace=True)

# Save CSV ---------------------------------------------------------------------
CSV_PATH = "elliptic_licit.csv"
metrics_df.to_csv(CSV_PATH, index=False)
logger.info("Saved metrics to %s", CSV_PATH)
